###FINAL

In [1]:
# Tahap 1: Instalasi Pustaka Python
# --------------------------------------------
!pip install -q gradio transformers torch bitsandbytes accelerate sentencepiece soundfile

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 MB 10.1 MB/s eta 0:00:00


In [2]:
# Tahap 2: Impor Pustaka
# -------------------------------------
import os
import torch
import gradio as gr
import numpy as np
import json
import re # --> PERBAIKAN 1: Menambahkan import 're' yang hilang
import getpass
import soundfile as sf # --> PERBAIKAN 2: Menambahkan import 'soundfile' untuk audio
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline, BitsAndBytesConfig, T5Tokenizer, T5ForConditionalGeneration

In [3]:
# Tahap 3: Pemuatan DUA Model AI
# --------------------------------------------------
print("Memuat model AI... Ini mungkin memakan waktu lebih lama.")
# 3.1 Model Speech-to-Text (Whisper)
transcriber = pipeline("automatic-speech-recognition", model="openai/whisper-base")
print("✅ Model Whisper (untuk suara) siap.")

# 3.2 Model Bahasa untuk PERCAKAPAN (DeepSeek)
model_name_chat = "deepseek-ai/deepseek-llm-7b-chat"
quant_config = BitsAndBytesConfig(load_in_4bit=True, bnb_4bit_quant_type="nf4", bnb_4bit_compute_dtype=torch.bfloat16)
tokenizer_chat = AutoTokenizer.from_pretrained(model_name_chat, trust_remote_code=True)
if tokenizer_chat.pad_token is None: tokenizer_chat.pad_token = tokenizer_chat.eos_token
model_chat = AutoModelForCausalLM.from_pretrained(model_name_chat, quantization_config=quant_config, device_map="auto")
llm_pipeline_chat = pipeline("text-generation", model=model_chat, tokenizer=tokenizer_chat, max_new_tokens=300)
print("✅ Model DeepSeek (untuk chat) siap.")

# 3.3 Model KHUSUS untuk GRAMMAR (Flan-T5)
model_name_grammar = "pszemraj/flan-t5-large-grammar-synthesis"
tokenizer_grammar = T5Tokenizer.from_pretrained(model_name_grammar)
model_grammar = T5ForConditionalGeneration.from_pretrained(model_name_grammar).to("cuda" if torch.cuda.is_available() else "cpu")
print("✅ Model Flan-T5 (untuk grammar) siap.")

Memuat model AI... Ini mungkin memakan waktu lebih lama.


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/290M [00:00<?, ?B/s]

generation_config.json: 0.00B [00:00, ?B/s]

...

tokenizer.json: 0.00B [00:00, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


config.json:   0%|          | 0.00/892 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.13G [00:00<?, ?B/s]

✅ Model Flan-T5 (untuk grammar) siap.


In [4]:
# Tahap 4: Fungsi Inti (Ditulis Ulang Sepenuhnya)
# ------------------------------------------------------------------

def analyze_grammar_with_specialist(text: str) -> str:
    """Menganalisis grammar dengan Flan-T5 (Sangat Andal)."""
    print(f"Analyzing grammar with T5 for: '{text}'")
    try:
        input_text = f"gec: {text}"
        input_ids = tokenizer_grammar(input_text, return_tensors="pt").input_ids.to(model_grammar.device)
        outputs = model_grammar.generate(input_ids, max_length=256, num_beams=4, early_stopping=True)
        corrected_sentence = tokenizer_grammar.decode(outputs[0], skip_special_tokens=True)
        if corrected_sentence.strip().lower() == text.strip().lower():
             return "<p style='font-family: sans-serif; color: green;'>✅ Tata bahasa Anda sudah benar!</p>"
        else:
            return f"<p style='font-family: sans-serif;'><strong>Saran Perbaikan:</strong> <span style='color: #4CAF50;'>{corrected_sentence}</span></p>"
    except Exception as e:
        return f"<p style='color: orange;'>Gagal menganalisis grammar: {e}</p>"

def detect_nlu_simple(text: str) -> str:
    """Mendeteksi Niat & Konteks dengan meminta Teks Biasa (Lebih Andal)."""
    print(f"Detecting NLU for: '{text}'")
    prompt = f"""As an NLU system, analyze the user's text. Identify the primary 'Intent' and a list of key 'Entities'.
Respond in two lines.
EXAMPLE:
USER TEXT: "I think the new F1 car from Ferrari is very fast."
YOUR RESPONSE:
Intent: Expressing Opinion
Entities: F1 car, Ferrari
---
USER TEXT: "{text}"
YOUR RESPONSE:
"""
    try:
        outputs = llm_pipeline_chat(prompt, do_sample=False, max_new_tokens=50)
        nlu_response = outputs[0]['generated_text'][len(prompt):].strip()
        # Format langsung ke HTML
        return nlu_response.replace("\n", "<br>")
    except Exception as e:
        print(f"NLU detection failed: {e}")
        return "Intent: N/A<br>Entities: N/A"

def process_conversation(text_input, audio_input, chat_history_messages):
    """Orkestrator utama dengan alur yang stabil dan terpisah."""
    user_text = ""
    # 1. Dapatkan Input
    if audio_input:
        try:
            audio_data, sample_rate = sf.read(audio_input)
            user_text = transcriber({"sampling_rate": sample_rate, "raw": audio_data})["text"].strip()
        except Exception as e: user_text = "_(Gagal memproses audio)_"
    elif text_input:
        user_text = text_input
    if not user_text: return chat_history_messages, "", ""

    chat_history_messages.append({"role": "user", "content": user_text})
    yield chat_history_messages, "<p>Menganalisis & Berpikir...</p>", ""

    # 2. Lakukan Analisis Secara Terpisah
    nlu_report = detect_nlu_simple(user_text)
    grammar_report = analyze_grammar_with_specialist(user_text)

    # 3. Gabungkan Laporan Analisis
    full_analysis_report = (
        f"<h3 style='font-family: sans-serif;'>Analisis Bahasa:</h3>"
        f"<div>{nlu_report}</div><hr>"
        f"<div>{grammar_report}</div>"
    )
    yield chat_history_messages, full_analysis_report, ""

    # 4. Hasilkan Jawaban Percakapan
    KONTEKS_MAKSIMAL = 10
    recent_history = chat_history_messages[-KONTEKS_MAKSIMAL:]
    system_prompt = "You are W11, a friendly and encouraging English tutor."
    messages_for_api = [{"role": "system", "content": system_prompt}] + recent_history
    prompt_text = tokenizer_chat.apply_chat_template(messages_for_api, tokenize=False, add_generation_prompt=True)

    outputs = llm_pipeline_chat(prompt_text, do_sample=True, temperature=0.7)
    clean_response = outputs[0]['generated_text'][len(prompt_text):].strip()

    # 5. Tampilkan Hasil Akhir
    chat_history_messages.append({"role": "assistant", "content": clean_response})
    yield chat_history_messages, full_analysis_report, ""


In [5]:
# Tahap 5: Membuat Antarmuka Gradio
# ---------------------------------------------------
with gr.Blocks(theme=gr.themes.Soft(), css="footer {display: none !important}") as demo:
    gr.Markdown("# 🤖 AI Speaking Tutor 🇺🇸")
    gr.Markdown("Ditenagai oleh AI terpisah untuk Percakapan dan Analisis Grammar.")
    with gr.Row():
        with gr.Column(scale=2):
            chatbot = gr.Chatbot(label="Conversation", height=500, type="messages", show_copy_button=True)
            with gr.Row():
                text_input = gr.Textbox(label="Input Teks", placeholder="Ketik pesan Anda di sini...", scale=4, container=False)
                mic_input = gr.Mic(label="Input Suara", type="filepath", scale=1)
        with gr.Column(scale=1):
            gr.Markdown("### 📝 Analisis Lengkap")
            analysis_output = gr.HTML(label="Feedback")

    text_input.submit(fn=process_conversation, inputs=[text_input, mic_input, chatbot], outputs=[chatbot, analysis_output, text_input])
    mic_input.stop_recording(fn=process_conversation, inputs=[text_input, mic_input, chatbot], outputs=[chatbot, analysis_output, text_input])

# Jalankan aplikasi
print("\n=======================================================================")
print("✅ Aplikasi Siap! Klik link di bawah ini untuk membuka antarmuka.")
print("=======================================================================")
demo.launch(share=True, debug=True)


✅ Aplikasi Siap! Klik link di bawah ini untuk membuka antarmuka.
Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://ae007261926ec60e52.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


Detecting NLU for: 'hello what is your name?'
Analyzing grammar with T5 for: 'hello what is your name?'
Detecting NLU for: 'i want to talk about the latest information about george russell that already extent his contract with mercedes'
Analyzing grammar with T5 for: 'i want to talk about the latest information about george russell that already extent his contract with mercedes'
Detecting NLU for: 'i want to know the driver standing in formula 1 in 2025'
Analyzing grammar with T5 for: 'i want to know the driver standing in formula 1 in 2025'
Detecting NLU for: 'who is in the top of the formula 1 driver champions in 2025 after singapore gp?'
Analyzing grammar with T5 for: 'who is in the top of the formula 1 driver champions in 2025 after singapore gp?'
Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://ae007261926ec60e52.gradio.live
